# Assignment: Segmenting and Clustering Neighborhoods in Toronto
Assignment Prompt:
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

## Importing relevant libraries

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
print ('Imported')

Imported


## Downloading, scraping, and cleaning data frame

In [16]:
# downloading data from webpage
req = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
tb = pd.DataFrame(df[0])
tb.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [19]:
# Ignore cells with a borough that is Not assigned.
tb = tb.drop(tb[(tb.Borough == "Not assigned")].index)

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
tb.Neighbourhood.replace("Not assigned", tb.Borough, inplace=True)
tb.Neighbourhood.fillna(tb.Borough, inplace=True)

# Drop duplicate entries after cleaning
tb = tb.drop_duplicates()

# Combine neighbourhood if under same postal code
df = pd.DataFrame({'Postcode':tb.Postcode.unique()})
df['Borough']=pd.DataFrame(list(set(tb['Borough'].loc[tb['Postcode'] == x['Postcode']])) for i, x in df.iterrows())
df['Neighborhood']=pd.Series(list(set(tb['Neighbourhood'].loc[tb['Postcode'] == x['Postcode']])) for i, x in df.iterrows())
df['Neighborhood']=df['Neighborhood'].apply(lambda x: ', '.join(x))
df.dtypes

df.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Ryerson, Garden District"
